# 소프트맥스 회귀(Softmax Regression)

---

### 1. 소프트맥스 회귀의 비용 함수 구현하기

In [1]:
import torch
import torch.nn.functional as F
torch.manual_seed(1)

In [2]:
z = torch.FloatTensor([1, 2, 3])

hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [7]:
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [26]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([3, 0, 1])


In [27]:
# 모든 원소가 0의 값을 가진 3 × 5 텐서 생성
y_one_hot = torch.zeros_like(hypothesis)
print(y)
print( y.unsqueeze(1))
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([3, 0, 1])
tensor([[3],
        [0],
        [1]])


tensor([[0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [29]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4614, grad_fn=<MeanBackward0>)


##### 1. F.softmax() + torch.log() = F.log_softmax()

In [32]:
print(F.softmax(z, dim=1))
print(torch.log(F.softmax(z, dim=1)))

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)
tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)


##### 2. F.log_softmax() + F.nll_loss() = F.cross_entropy()

In [31]:
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4614, grad_fn=<NllLossBackward>)

### 2.소프트맥스 회귀 구현하기

##### low-level로 구현 하기

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [34]:
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


In [40]:
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(3, requires_grad=True)

optimizer = optim.SGD([W, b], lr = 1e-5)

In [36]:
y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_one_hot)

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])


In [42]:
nb_epoch = 2000
for epoch in range(nb_epoch + 1):
    hypothesis = F.softmax(torch.matmul(x_train, W) + b)
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epoch, cost.item()
        ))

c:\users\bj_park\anaconda3\envs\nlp_env\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch    0/2000 Cost: 1.098592
Epoch  100/2000 Cost: 1.096594
Epoch  200/2000 Cost: 1.094659
Epoch  300/2000 Cost: 1.092785
Epoch  400/2000 Cost: 1.090968
Epoch  500/2000 Cost: 1.089207
Epoch  600/2000 Cost: 1.087498
Epoch  700/2000 Cost: 1.085840
Epoch  800/2000 Cost: 1.084230
Epoch  900/2000 Cost: 1.082666
Epoch 1000/2000 Cost: 1.081146
Epoch 1100/2000 Cost: 1.079669
Epoch 1200/2000 Cost: 1.078231
Epoch 1300/2000 Cost: 1.076832
Epoch 1400/2000 Cost: 1.075470
Epoch 1500/2000 Cost: 1.074144
Epoch 1600/2000 Cost: 1.072851
Epoch 1700/2000 Cost: 1.071591
Epoch 1800/2000 Cost: 1.070362
Epoch 1900/2000 Cost: 1.069162
Epoch 2000/2000 Cost: 1.067992


##### 하이레벨로 구현하기

In [43]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433506
Epoch 1000/1000 Cost: 0.399962
